In [ ]:
import numpy as np

In [ ]:
images = np.load('reordered_images.npy')
images = np.transpose(images, (0, 3, 1, 2))
images = images[:9500]

In [ ]:
images.shape

(9500, 3, 32, 32)

In [ ]:
features = np.load('feature_representations_sorted_9500.npy')
features = features[:9500]

In [ ]:
features = np.load('feature_representations_sorted_9500.npy')
features = features[:9500]
std_devs_per_image = np.std(features, axis=1)
features = np.mean(features, 1)
print(features.shape)
print(std_devs_per_image.shape)

(9500, 512)
(9500, 512)


In [ ]:
print(features.shape)
print(std_devs_per_image.shape)

(9500, 512)
(9500, 512)


In [ ]:
data_format='channels_last'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, Dropout, RandomFlip, RandomRotation, RandomZoom, GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1_l2
l1_reg = 0.0001
l2_reg = 0.0001
def resnet_block(input_tensor, filters, kernel_size, strides=(1, 1), activate_before_residual=False):
    x = input_tensor

    if activate_before_residual:
        x = BatchNormalization()(x)
        x = ReLU()(x)
        input_tensor = x

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding="same", data_format=data_format ,kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=1, padding="same", data_format=data_format,kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))(x)
    x = BatchNormalization()(x)

    if strides != (1, 1) or activate_before_residual:
        input_tensor = Conv2D(filters=filters, kernel_size=1, strides=strides, padding="same", data_format=data_format, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))(input_tensor)
        input_tensor = BatchNormalization()(input_tensor)

    x = Add()([x, input_tensor])
    x = ReLU()(x)

    return x


def build_submodel(input_shape=(3, 32, 32), num_blocks=6, output_size=512, channels=64):
    inputs = Input(shape=input_shape)
    x = inputs
    x = x/255
    #x = tf.transpose(x, (0, 2, 3, 1))
    x = tf.keras.layers.Permute((2, 3, 1))(x)
    # Initial convolutional layer
    x = Conv2D(channels, (3, 3), padding='same', data_format=data_format)(x)

    # Adding ResNet blocks
    for i in range(num_blocks):
        strides = (1, 1) if i == 0 else (2, 2)  # Downsample on first block of each feature map size
        x = resnet_block(x, filters=channels * (2 ** (i // 2)), kernel_size=3, strides=strides, activate_before_residual=(i == 0))

    # Global average pooling and output layer
    x = GlobalAveragePooling2D(data_format=data_format)(x)
    x = Dropout(0.5)(x)
    outputs = Dense(output_size, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs)

    return model

# Build the model
submodel = build_submodel()

def build_model(input_shape=(3, 32, 32)):
    inputs = Input(shape=input_shape)
    x = inputs
    x = tf.keras.layers.Permute((2, 3, 1))(x)
    x = RandomFlip("horizontal_and_vertical")(x)
    x = RandomRotation(0.2)(x)
    x = RandomZoom(0.2, 0.2)(x)
    x = tf.keras.layers.Permute((3, 1, 2))(x)
    noise_sigmas = Input(shape=(512,))
    x = submodel(x)
    outputs = x
    noisy_versions = []
    for _ in range(5):  # Sample noise 5 times
        noise = tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=noise_sigmas, dtype=tf.float32)
        noisy_versions.append(noise)
    noisy_versions_tensor = tf.stack(noisy_versions, axis=0)
    averaged_noise = tf.reduce_mean(noisy_versions_tensor, axis=0)
    outputs = outputs + averaged_noise
    outputs = tf.keras.layers.Identity()(outputs)
    model = Model(inputs=[inputs, noise_sigmas], outputs=outputs)
    return model




model = build_model()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0), loss='mse')

In [ ]:
def cosine_schedule(progress, total, max_lr=0.0009):
    if progress < 0.1:
        #progress /= 0.1
        #lr = max_lr*progress
        lr = max_lr + 0.0001
    else:
        progress = (progress - 0.1) / 0.9
        lr = max_lr * 0.5 * (1 + math.cos(progress * math.pi))
        lr += 0.0001
    return lr

def get_lr(step, budget, start):
    current_time = time.time()
    elapsed_time = current_time - start
    steps_per_second = step / elapsed_time
    total_estimated_steps = steps_per_second * budget
    progress = step / total_estimated_steps

    lr = cosine_schedule(progress, total_estimated_steps)
    return lr

In [ ]:
import math
class LRCallback(tf.keras.callbacks.Callback):
    def __init__(self, budget):
        super().__init__()
        self.budget = budget
        self.start = None

    def on_batch_end(self, batch, logs=None):
        step = self.model.optimizer.iterations.numpy()
        if step == 1:
            self.start = time.time()
        if time.time() - self.start > self.budget:
            print("Training budget exceeded. Stopping training.")
            self.model.stop_training = True
            return
        lr = get_lr(step, self.budget, self.start)
        self.model.optimizer.learning_rate.assign(lr)

In [ ]:
import time
budget = 3600*0.1
model.fit(x=[images, std_devs_per_image], y=features, epochs=1000, batch_size=64,callbacks=[LRCallback(budget)])

Epoch 1/1000
149/149 [==============================] - 8s 12ms/step - loss: 1.0509
Epoch 2/1000
149/149 [==============================] - 2s 13ms/step - loss: 0.6835
Epoch 3/1000
149/149 [==============================] - 2s 10ms/step - loss: 0.6051
Epoch 4/1000
149/149 [==============================] - 2s 10ms/step - loss: 0.5619
Epoch 5/1000
149/149 [==============================] - 2s 11ms/step - loss: 0.5364
Epoch 6/1000
149/149 [==============================] - 2s 10ms/step - loss: 0.5158
Epoch 7/1000
149/149 [==============================] - 2s 10ms/step - loss: 0.5034
Epoch 8/1000
149/149 [==============================] - 2s 10ms/step - loss: 0.4854
Epoch 9/1000
149/149 [==============================] - 2s 10ms/step - loss: 0.4776
Epoch 10/1000
149/149 [==============================] - 2s 11ms/step - loss: 0.4784
Epoch 11/1000
149/149 [==============================] - 2s 11ms/step - loss: 0.4698
Epoch 12/1000
149/149 [==============================] - 2s 10ms/step - lo

In [ ]:
!pip install onnx tf2onnx

In [ ]:
import tf2onnx
import onnx
input_signature = (tf.TensorSpec((1, 3, 32, 32), tf.float32, name="x"),)
onnx_model, _ = tf2onnx.convert.from_keras(submodel,input_signature)
onnx.save(onnx_model, "/kaggle/working/submodel28.onnx")

In [ ]:
import requests
def model_stealing_submit(path_to_onnx_file: str):
    endpoint = "/modelstealing/submit"
    SERVER_URL = "http://34.71.138.79:9090"
    TEAM_TOKEN = "5KogzTO5QjSdXupe"
    url = SERVER_URL + endpoint
    with open(path_to_onnx_file, "rb") as f:
        response = requests.post(url, files={"file": f}, headers={"token": TEAM_TOKEN})
        if response.status_code == 200:
            print("Request ok")
            print(response.json())
        else:
            raise Exception(
                f"Model stealing submit failed. Code: {response.status_code}, content: {response.json()}"
            )

model_stealing_submit("/kaggle/working/submodel28.onnx")

Request ok
{'score': 24.736305236816406}
